# Assignment: Video Quality Inference

To this point in the class, you have learned various techniques for leading and analyzing packet captures of various types, generating features from those packet captures, and training and evaluating models using those features.

In this assignment, you will put all of this together, using a network traffic trace to train a model to automatically infer video quality of experience from a labeled traffic trace.

## Part 1: Warmup

The first part of this assignment builds directly on the hands-on activities but extends them slightly.

### Extract Features from the Network Traffic

Load the `netflix.pcap` file, which is a packet trace that includes network traffic. 

Click [here](https://github.com/noise-lab/ml-systems/blob/main/docs/notebooks/data/netflix.pcap) to download `netflix.pcap`.


In [1]:
import pandas as pd
import numpy as np
from scapy.all import *
from netml.pparser.parser import PCAP

netflix_packets = rdpcap('netflix.pcap')

print(netflix_packets)

<netflix.pcap: TCP:141143 UDP:232 ICMP:16 Other:80>


In [ ]:
for i in range(5):
    print(netflix_packets[i].summary())
    print(netflix_packets[i].layers())

#layers: Ether, IP, UDP, DNS Qry

Ether / IP / UDP / DNS Qry "b'fonts.gstatic.com.'" 
[<class 'scapy.layers.l2.Ether'>, <class 'scapy.layers.inet.IP'>, <class 'scapy.layers.inet.UDP'>, <class 'scapy.layers.dns.DNS'>]
Ether / IP / UDP / DNS Qry "b'fonts.gstatic.com.'" 
[<class 'scapy.layers.l2.Ether'>, <class 'scapy.layers.inet.IP'>, <class 'scapy.layers.inet.UDP'>, <class 'scapy.layers.dns.DNS'>]
Ether / IP / UDP / DNS Qry "b'googleads.g.doubleclick.net.'" 
[<class 'scapy.layers.l2.Ether'>, <class 'scapy.layers.inet.IP'>, <class 'scapy.layers.inet.UDP'>, <class 'scapy.layers.dns.DNS'>]
Ether / IP / UDP / DNS Qry "b'googleads.g.doubleclick.net.'" 
[<class 'scapy.layers.l2.Ether'>, <class 'scapy.layers.inet.IP'>, <class 'scapy.layers.inet.UDP'>, <class 'scapy.layers.dns.DNS'>]
Ether / IP / UDP / DNS Qry "b'ytimg.l.google.com.'" 
[<class 'scapy.layers.l2.Ether'>, <class 'scapy.layers.inet.IP'>, <class 'scapy.layers.inet.UDP'>, <class 'scapy.layers.dns.DNS'>]


### Identifying the Service Type

Use the DNS traffic to filter the packet trace for Netflix traffic.

In [ ]:
test = str(netflix_packets[1])
#if netflix_packets[1].contains('DNS Qry'):
if 'DNS' in test:
    print("yes")

yes


In [19]:
netflix_domains = ['netflix.com', 'netflix.net', 'nflxso.net', 'nflxvideo.net', 'nflxext.com' 'nflximg.com', 'nflximg.net']
#source: https://www.netify.ai/resources/applications/netflix

netflix_traffic = []
#do domains in a bit

netflix_traffic_ips = []

for packet in netflix_packets:
    packet_string_form = str(packet)

    #if 'DNS Qry' in packet_string_form:
    if 'DNS' in packet_string_form:
        packet_summary = packet.summary()
        #netflix_traffic.append(packet_summary)

        for domain in netflix_domains:
            #if ('netflix.com' in packet_summary) | ('netflix.net' in packet_summary) | ('nflxvideo.net' in packet_summary):
            if domain in packet_summary:
                if 'DNS Ans' in packet_summary:
                    print("TODO: IP addresses??????")
                netflix_traffic.append(packet_summary)

print(netflix_traffic)

['Ether / IP / UDP / DNS Qry "b\'www.netflix.com.\'" ', 'Ether / IP / UDP / DNS Qry "b\'customerevents.netflix.com.\'" ', 'Ether / IP / UDP / DNS Qry "b\'ipv4-c001-cdg001-ix.1.oca.nflxvideo.net.\'" ', 'Ether / IP / UDP / DNS Qry "b\'ipv4-c024-cdg001-ix.1.oca.nflxvideo.net.\'" ', 'Ether / IP / UDP / DNS Qry "b\'ipv4-c072-cdg001-ix.1.oca.nflxvideo.net.\'" ', 'Ether / IP / UDP / DNS Qry "b\'occ-0-56-55.1.nflxso.net.\'" ', 'Ether / IP / UDP / DNS Qry "b\'tp-s.nflximg.net.\'" ', 'Ether / IP / UDP / DNS Qry "b\'push.prod.netflix.com.\'" ', 'Ether / IP / UDP / DNS Qry "b\'ipv4-c005-cdg001-ix.1.oca.nflxvideo.net.\'" ', 'Ether / IP / UDP / DNS Qry "b\'ipv4-c069-cdg001-ix.1.oca.nflxvideo.net.\'" ', 'Ether / IP / UDP / DNS Qry "b\'ipv4-c071-cdg001-ix.1.oca.nflxvideo.net.\'" ', 'Ether / IP / UDP / DNS Qry "b\'ipv4-c063-cdg001-ix.1.oca.nflxvideo.net.\'" ', 'Ether / IP / UDP / DNS Qry "b\'www.netflix.com.\'" ', 'Ether / IP / UDP / DNS Qry "b\'www.netflix.com.\'" ', 'Ether / IP / UDP / DNS Qry "b\'ww

### Generate Statistics

Generate statistics and features for the Netflix traffic flows. Use the `netml` library or any other technique that you choose to generate a set of features that you think would be good features for your model. 

**Write a brief justification for the features that you have chosen.**

### Inferring Segment downloads

In addition to the features that you could generate using the `netml` library or similar, add to your feature vector a "segment downloads rate" feature, which indicates the number of video segments downloaded for a given time window.

Note: If you are using the `netml` library, generating features with `SAMP` style options may be useful, as this option gives you time windows, and you can then simply add the segment download rate to that existing dataframe.

## Part 2: Video Quality Inference

You will now load the complete video dataset from a previous study to train and test models based on these features to automatically infer the quality of a streaming video flow.

For this part of the assignment, you will need two pickle files, which we provide for you by running the code below:

```

!gdown 'https://drive.google.com/uc?id=1N-Cf4dJ3fpak_AWgO05Fopq_XPYLVqdS' -O netflix_session.pkl
!gdown 'https://drive.google.com/uc?id=1PHvEID7My6VZXZveCpQYy3lMo9RvMNTI' -O video_dataset.pkl

```

### Load the File

Load the video dataset pickle file.

### Clean the File

1. The dataset contains video resolutions that are not valid. Remove entries in the dataset that do not contain a valid video resolution. Valid resolutions are 240, 360, 480, 720, 1080.

2. The file also contains columns that are unnecessary (in fact, unhelpful!) for performing predictions. Identify those columns, and remove them.

**Briefly explain why you removed those columns.**

### Prepare Your Data

Prepare your data matrix, determine your features and labels, and perform a train-test split on your data.

### Train and Tune Your Model

1. Select a model of your choice.
2. Train the model using your training data.

### Tune Your Model

Perform hyperparameter tuning to find optimal parameters for your model.

### Evaluate Your Model

Evaluate your model accuracy according to the following metrics:

1. Accuracy
2. F1 Score
3. Confusion Matrix
4. ROC/AUC

## Part 3: Predict the Ongoing Resolution of a Real Netflix Session

Now that you have your model, it's time to put it in practice!

Use a preprocessed Netflix video session to infer **and plot** the resolution at 10-second time intervals.